# use data augmentation when we have small dataset,small invariance,reduce overfit

In [8]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D

In [2]:
from keras.preprocessing.image import ImageDataGenerator  # preprocess image by class ImageDataGenerator-->to generate multiple image in train,not generating image in test ,validation
#import keras.utils as image
import matplotlib.pyplot as plt
import cv2

In [3]:

train_data=ImageDataGenerator(  # ImageDataGenerator have invariance such as rescale etc
rescale=1./255,  # normalization  # transformation technique
zoom_range=0.2,   #<1-zoom in,>1-zoom out
shear_range=0.2,  
rotation_range=20,
)

# above code generate ImageDataGenerator object
# below is main generator

train_g=train_data.flow_from_directory(   # flow used for single image only,if u have multiple image in directory use flow_from_directory
r"D:\deep learning\mask\archive (4)\data\ds\train",
target_size=(200,200),  # change all image size to 200,200 ,generate image in train
batch_size=16,   # each batch have 16 generated image
class_mode="categorical")   # if binary-no need to convert to one hot ,use sigmoid,if muticlass-categorical, use softmax


# generator send transformed image to ur model at training time

validation_data=ImageDataGenerator(rescale=1./255)  # in validation,test only using rescaling

validation_gen=validation_data.flow_from_directory(
r"D:\deep learning\mask\archive (4)\data\ds\validation",
target_size=(200,200),
batch_size=16,
class_mode="categorical")

Found 5000 images belonging to 2 classes.
Found 2553 images belonging to 2 classes.


In [ ]:
# whenevr u do transformation ur image is missing in some portion for that fill with nearest ie fill_mode="nearest"

In [11]:
model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=(200,200,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation="relu"))

model.add(Dense(2,activation="softmax"))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 198, 198, 200)     5600      
                                                                 
 activation_2 (Activation)   (None, 198, 198, 200)     0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 99, 99, 200)      0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 97, 97, 100)       180100    
                                                                 
 activation_3 (Activation)   (None, 97, 97, 100)       0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 48, 48, 100)      0         
 2D)                                                  

In [ ]:
# 0 indicates give more sparcity to our dataset

In [ ]:
model.fit_generator(train_g,  # at training take image and generate image ie augmentation image,this generator 
steps_per_epoch=5000//16,  # train size//batch size  ie 312 step to complete one epoch
epochs=3,  # if epochs more, more information we can extract
validation_data=validation_gen,
validation_steps=2553//16)

C:\Users\SSS\AppData\Local\Temp\ipykernel_5320\3903969976.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_g,  # at training take image and generate image ie augmentation image,this generator


Epoch 1/3
 15/312 [>.............................] - ETA: 1:08:20 - loss: 1.4456 - accuracy: 0.5708

In [ ]:
# in new version no need to give fit_generator